# Est-ce que courrir à domicile affecte les performances des pilotes ? 

In [17]:
import pandas as pd
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))
from options.config import DATA_DIR

drivers = pd.read_csv(DATA_DIR/"drivers.csv")
results = pd.read_csv(DATA_DIR / "results.csv")
races = pd.read_csv(DATA_DIR/"races.csv")
drivers["nationality"] = drivers["nationality"].apply(lambda x: x.lower())
drivers


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,british,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,german,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,german,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
854,856,de_vries,21,DEV,Nyck,de Vries,1995-02-06,dutch,http://en.wikipedia.org/wiki/Nyck_de_Vries
855,857,piastri,81,PIA,Oscar,Piastri,2001-04-06,australian,http://en.wikipedia.org/wiki/Oscar_Piastri
856,858,sargeant,2,SAR,Logan,Sargeant,2000-12-31,american,http://en.wikipedia.org/wiki/Logan_Sargeant
857,859,lawson,40,LAW,Liam,Lawson,2002-02-11,new zealander,http://en.wikipedia.org/wiki/Liam_Lawson


In [2]:
races["country"] = races["name"].apply(lambda x: x.split(" ")[0].lower())


In [3]:
# Calcule de la position moyenne par pilote
position_moyenne = results.groupby("driverId").agg(moyenne_overall= ("positionOrder", "mean")).sort_values("moyenne_overall")
position_moyenne_pilote = drivers.merge(position_moyenne, on="driverId", how="inner")


In [4]:
# Calcule de la position moyenne des pilotes lors des courses dans leurs pays
local_races = races.merge(drivers, left_on="country", right_on = "nationality")
local_races_moyenne = local_races.merge(results, on =[ "raceId", "driverId"]).groupby("driverId").agg(moyenne_locale =("positionOrder","mean"))
local_races_moyenne


,moyenne_locale
driverId,
1,3.000000
2,13.300000
3,8.111111
4,7.857143
6,15.000000
...,...
849,16.000000
852,11.666667
853,18.000000


In [ ]:
compare = pd.merge(local_races_moyenne,position_moyenne_pilote, on="driverId")
compare[["driverId","forename","surname","moyenne_overall","moyenne_locale","nationality"]].query("nationality == 'japanase'")


,driverId,forename,surname,moyenne_overall,moyenne_locale,nationality
0,1,Lewis,Hamilton,4.950581,3.000000,british
10,14,David,Coulthard,9.485830,8.733333,british
14,18,Jenson,Button,9.695793,10.647059,british
15,19,Anthony,Davidson,16.166667,18.000000,british
37,51,Justin,Wilson,14.562500,16.000000,british
...,...,...,...,...,...,...
465,820,Max,Chilton,17.685714,16.500000,british
468,829,Will,Stevens,16.631579,13.000000,british
473,835,Jolyon,Palmer,15.270270,19.000000,british
481,846,Lando,Norris,8.275862,5.166667,british


In [29]:
# Calcule de la position moyenne des pilotes lors des courses dans des pays etrangers à leur pays d'origine
# Normalisation des colonnes pour éviter les problèmes de casse
drivers["nationality"] = drivers["nationality"].str.lower()
races["country"] = races["name"].str.split(" ").str[0].str.lower()

# Merge des données pour relier les pilotes, les résultats et les courses
merged_data = results.merge(drivers, on="driverId").merge(races, on="raceId")

# Filtrer les courses dans des pays étrangers (nationalité différente du pays de la course)
foreign_races = merged_data[merged_data["nationality"] != merged_data["country"]]

# Calcul de la position moyenne des pilotes dans des pays étrangers
average_position_foreign = (
    foreign_races.groupby("driverId")["positionOrder"]
    .mean()
    .reset_index()
    .rename(columns={"positionOrder": "average_position_foreign"})
)

# Ajout des noms des pilotes pour plus de lisibilité
average_position_foreign = average_position_foreign.merge(
    drivers[["driverId", "surname", "forename"]], on="driverId"
)

average_position_foreign.head()
compare  = pd.merge(compare,average_position_foreign, on = "driverId")
compare.head()


MergeError: Passing 'suffixes' which cause duplicate columns {'surname_x', 'forename_x'} is not allowed.